In [26]:
import numpy as np
import pandas as pd

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en.stop_words import STOP_WORDS
import en_core_web_lg

from tqdm import tqdm_notebook as tqdm
from pprint import pprint

In [67]:
doc = pd.read_csv (r'test_agor.csv')
newest_doc = doc[['Body']] 
newest_doc.head()
newest_doc.Body.tolist()[4]

"Hey guys writing after a long time...... I would like to share my current life and agoraphobia condition..... My anxiety and panic attacks have faded away due to medication and staying restricted in my safe place........Even not having a panic attack since 4-5 months I am unable to go out anywhere I have made up mind that if I go out I will suffer though I am sure that everything will be perfectly but still I am unable to take a step I have got myself too comfortable in my safe. Place where every thing is according to me and under controlled conditions\n\nI don't think that the way I am living can work for longer some day or other I have to take steps but I am afraid and started to have suicidal thoughts\n\n( Sorry for my English )\n\nI woul like to add that I have dropped out of college the worst decision in my life I know I am going to regret it afterwards like 2-5 years from now .."

In [49]:
nlp= spacy.load("en")

# My list of stop words.
stop_list = ["Agoraphobia","Agoraphobic", "agoraphobia", "agoraphobic"]

# Updates spaCy's default stop words list with my additional words. 
nlp.Defaults.stop_words.update(stop_list)

# Iterates over the words in the stop words list and resets the "is_stop" flag.
for word in STOP_WORDS:
    lexeme = nlp.vocab[word]
    lexeme.is_stop = True

In [50]:
def lemmatizer(doc):
    # This takes in a doc of tokens from the NER and lemmatizes them. 
    # Pronouns (like "I" and "you" get lemmatized to '-PRON-', so I'm removing those.
    doc = [token.lemma_ for token in doc if token.lemma_ != '-PRON-']
    doc = u' '.join(doc)
    return nlp.make_doc(doc)
    
def remove_stopwords(doc):
    # This will remove stopwords and punctuation.
    # Use token.text to return strings, which we'll need for Gensim.
    doc = [token.text for token in doc if token.is_stop != True and token.is_punct != True]
    return doc

# The add_pipe function appends our functions to the default pipeline.
nlp.add_pipe(lemmatizer,name='lemmatizer',after='ner')
nlp.add_pipe(remove_stopwords, name="stopwords", last=True)

In [69]:
doc_list = []
# Iterates through each article in the corpus.
for doc in newest_doc.Body.tolist():
    # Passes that article through the pipeline and adds to a new list.
    pr = nlp(str(doc))
    doc_list.append(pr)

In [70]:
doc_list

[['nan'],
 ['slowly',
  'fall',
  'start',
  'university',
  'feel',
  'like',
  'belong',
  'like',
  'trick',
  'way',
  'catch',
  'day',
  'feel',
  'like',
  'stare',
  'know',
  'true',
  'special',
  'past',
  'year',
  'struggle',
  'baseline',
  'depression',
  'strong',
  'forgot',
  'love',
  'cloud',
  'finally',
  'sunlight',
  'week',
  'plunge',
  'depressive',
  'episode',
  'strong',
  'pretty',
  'lie',
  'bed',
  'day',
  'day',
  'week',
  'Christmas',
  'break',
  'leave',
  'house',
  'stare',
  'like',
  'magnet',
  'hold',
  'bed',
  'pull',
  'away',
  'magnet',
  'frightened',
  'body',
  'heart',
  'suffer',
  'like',
  'sit',
  'living',
  'room',
  'anymore',
  'open',
  'live',
  'small',
  'apartment',
  'bed',
  'safe',
  '\n\n ',
  'night',
  'new',
  'year',
  'eve',
  'partner',
  'party',
  'follow',
  'far',
  'away',
  'denial',
  'possibly',
  'condition',
  'obviously',
  'whiner',
  'resolve',
  'parent',
  '\n\n ',
  'try',
  'leave',
  'stand'

In [71]:
# Creates, which is a mapping of word IDs to words.
words = corpora.Dictionary(doc_list)

# Turns each document into a bag of words.
corpus = [words.doc2bow(doc) for doc in doc_list]

In [72]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=words,
                                           num_topics=10, 
                                           random_state=2,
                                           update_every=1,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [74]:
#print the keyword in the 10 topics
pprint(lda_model.print_topics(num_words=100))

[(0,
  '0.018*"\n'
  '\n'
  ' " + 0.013*"feel" + 0.013*"like" + 0.013*"people" + 0.012*"help" + '
  '0.011*"want" + 0.011*"time" + 0.011*"know" + 0.011*"anxiety" + '
  '0.009*"leave" + 0.008*"work" + 0.008*"\n'
  ' " + 0.007*"think" + 0.007*"house" + 0.007*"year" + 0.006*"need" + '
  '0.006*"start" + 0.006*"come" + 0.005*"school" + 0.005*"panic" + '
  '0.005*"talk" + 0.005*"try" + 0.005*"day" + 0.005*"good" + 0.004*"thing" + '
  '0.004*"stay" + 0.004*"outside" + 0.004*"friend" + 0.004*"attack" + '
  '0.004*"home" + 0.004*"store" + 0.003*"social" + 0.003*"fear" + '
  '0.003*"place" + 0.003*"right" + 0.003*"advice" + 0.003*"find" + '
  '0.003*"able" + 0.003*"thank" + 0.003*"job" + 0.003*"hard" + 0.003*"  " + '
  '0.003*"look" + 0.003*"grocery" + 0.003*"lot" + 0.003*"etc" + 0.003*"guy" + '
  '0.003*"use" + 0.003*"boyfriend" + 0.003*"live" + 0.003*"issue" + '
  '0.003*"way" + 0.003*"experience" + 0.003*"understand" + 0.003*"walk" + '
  '0.003*"family" + 0.003*"mental" + 0.003*"class" + 0.0